### ResNet

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train, X_test = X_train/255.0, X_test/255.0 # 정규화
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10) # 원-핫 인코딩

In [ ]:
# 1. 일반 CNN 모델 정의
def build_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
        layers.MaxPool2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPool2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPool2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# 2. VGG16 모델 (Pretrained 사용)
def build_vgg16():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))
    base_model.trainable = False # 기존 가중치 동결(Fine-tuning 가능)
    
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# 3. ResNet50 모델 (Pretrained 사용)
def build_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32,32,3))
    base_model.trainable = False # 기존 가중치 동결
    
    model = models.Sequential([
        base_model,
        layers.GlobalMaxPooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# 모델 학습 및 평가
def train_and_evaluate(model, name, epochs=5):
    print(f'\nTraining {name}...')
    model.fit(X_train, y_train, epochs=epochs, batch_size=128, validation_data=(X_test,y_test), verbose=2)
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f'{name} Test Accuracy : {test_acc:.4f}')
    return test_acc

In [8]:
# 실행
cnn_model = build_cnn()
vgg16_model = build_vgg16()
resnet50_model = build_resnet50()

cnn_acc = train_and_evaluate(cnn_model, "CNN")
vgg_acc = train_and_evaluate(vgg16_model, 'VGG16')
res_acc = train_and_evaluate(resnet50_model, "ResNet50")

# 최종 결과 출력
print(f"\nFinal Accuracy Comparison :")
print(f"CNN Accuracy : {cnn_acc:.4f}")
print(f"VGG16 Accuracy : {vgg_acc:.4f}")
print(f"ResNet50 Accuracy : {res_acc:.4f}")


c:\Users\main\miniconda3\envs\dl\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step

Training CNN...
Epoch 1/5
391/391 - 25s - 65ms/step - accuracy: 0.4514 - loss: 1.5166 - val_accuracy: 0.5826 - val_loss: 1.1746
Epoch 2/5
391/391 - 21s - 55ms/step - accuracy: 0.6157 - loss: 1.0916 - val_accuracy: 0.6413 - val_loss: 1.0126
Epoch 3/5
391/391 - 20s - 52ms/step - accuracy: 0.6787 - loss: 0.9218 - val_accuracy: 0.6878 - val_loss: 0.9148
Epoch 4/5
391/391 - 19s - 48ms/step - accuracy: 0.7125 - loss: 0.8269 - val_accuracy: 0.7141 - val_loss: 0.8304
Epoch 5/5
391/391 - 22s - 55ms/step - accuracy: 0.7445 - loss: 0.7326 - val_accuracy: 0.6988 - val_loss: 0.8730
CNN Test Accuracy : 0.6988

Training VGG16...
Epoch 1/5
391/391 - 171s - 438ms/step - accuracy: 0.5106 - loss: 1.4128 - val_accuracy: 0.5577 - val_loss: 1.2721
Epoch 2/5
391/391 - 174s - 445ms/step - accuracy: 0.5830 - loss: 1.2040 - val_accuracy: 0.5744 - val_loss: 1.2098
Epoch 3/5
391/391 - 162s - 415ms/step - accurac